# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,9.80,86,100,1.86,CL,1706492162
1,1,longyearbyen,78.2186,15.6401,-11.09,79,0,5.14,SJ,1706492351
2,2,bredasdorp,-34.5322,20.0403,18.94,81,100,1.33,ZA,1706492351
3,3,adamstown,-25.0660,-130.1015,24.51,77,62,4.50,PN,1706492351
4,4,grytviken,-54.2811,-36.5092,4.86,87,22,5.42,GS,1706492351


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.7
)

# Display the map
city_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 10) &
                                (city_data_df["Humidity"] > 50) & (city_data_df["Humidity"] < 75) &
                                (city_data_df["Cloudiness"] <= 50) & (city_data_df["Wind Speed"] < 2)]

# Drop any rows with null values
ideal_cities = ideal_cities.dropna(how='any')

# Display sample data
ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
97,97,tura,25.5198,90.2201,15.26,55,1,0.81,IN,1706492364
121,121,porto novo,6.4965,2.6036,26.90,61,16,1.58,BJ,1706492368
215,215,maharajgani,26.1167,84.4833,10.13,54,18,0.54,IN,1706492382
250,250,luxor,25.6989,32.6421,10.03,53,0,1.03,EG,1706492386
274,274,ponta delgada,37.7333,-25.6667,12.40,71,40,1.54,PT,1706492390
323,323,fale old settlement,-9.3852,-171.2468,29.55,69,25,1.80,TK,1706492396
340,340,oroville east,39.5113,-121.4752,18.29,72,0,0.00,US,1706492399
371,371,kruisfontein,-34.0033,24.7314,17.16,71,30,0.32,ZA,1706492405
417,417,senanga,-16.1167,23.2667,19.54,71,3,1.91,ZM,1706492412
560,560,pathein,16.7833,94.7333,23.99,59,8,1.63,MM,1706492437


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
97,tura,IN,25.5198,90.2201,55,
121,porto novo,BJ,6.4965,2.6036,61,
215,maharajgani,IN,26.1167,84.4833,54,
250,luxor,EG,25.6989,32.6421,53,
274,ponta delgada,PT,37.7333,-25.6667,71,
323,fale old settlement,TK,-9.3852,-171.2468,69,
340,oroville east,US,39.5113,-121.4752,72,
371,kruisfontein,ZA,-34.0033,24.7314,71,
417,senanga,ZM,-16.1167,23.2667,71,
560,pathein,MM,16.7833,94.7333,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tura - nearest hotel: No hotel found
porto novo - nearest hotel: Hôtel 6500 CFA
maharajgani - nearest hotel: No hotel found
luxor - nearest hotel: Salam
ponta delgada - nearest hotel: Hotel Talisman
fale old settlement - nearest hotel: No hotel found
oroville east - nearest hotel: Feather Falls Casino & Lodge
kruisfontein - nearest hotel: No hotel found
senanga - nearest hotel: Senanga Safari Lodge
pathein - nearest hotel: Taan Taan Ta


,City,Country,Lat,Lng,Humidity,Hotel Name
97,tura,IN,25.5198,90.2201,55,No hotel found
121,porto novo,BJ,6.4965,2.6036,61,Hôtel 6500 CFA
215,maharajgani,IN,26.1167,84.4833,54,No hotel found
250,luxor,EG,25.6989,32.6421,53,Salam
274,ponta delgada,PT,37.7333,-25.6667,71,Hotel Talisman
323,fale old settlement,TK,-9.3852,-171.2468,69,No hotel found
340,oroville east,US,39.5113,-121.4752,72,Feather Falls Casino & Lodge
371,kruisfontein,ZA,-34.0033,24.7314,71,No hotel found
417,senanga,ZM,-16.1167,23.2667,71,Senanga Safari Lodge
560,pathein,MM,16.7833,94.7333,59,Taan Taan Ta


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.7,
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)